In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
from apps.reactions.replicawork import ReplicaWorkchain
from aiida.work.process import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.folder import FolderData

from aiida.common.exceptions import NotExistent
from aiida.orm.data.base import Str

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

import matplotlib.pyplot as plt
from pprint import pprint

import ase.io
import tempfile

import re
import numpy as np
import scipy.constants

from collections import OrderedDict

from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return '<img width="400px" src="data:image/png;base64,{}" title="">'.format(th)
def html_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

In [ ]:
def replicas_from_restart(restart_file_path):
    content = open(restart_file_path).read()
    m = re.search(r'\n\s*&CELL\n(.*?)\n\s*&END CELL\n', content, re.DOTALL)
    cell_lines = [line.strip().split() for line in m.group(1).split("\n")]
    cell_str = [line[1:] for line in cell_lines if line[0] in 'ABC']
    cell = np.array(cell_str, np.float64)
    
    structure_data_list = []
    
    matches = re.findall(r'\n\s*&COORD\n(.*?)\n\s*&END COORD\n', content, re.DOTALL)

    coord_line_sets = [
        [line.strip().split() for line in m.split("\n")] for m in matches
    ]
    coord_set_with_elements = coord_line_sets[-1]
    replica_coord_line_sets = coord_line_sets[:-1]
    element_list = [line[0] for line in coord_set_with_elements]
    for i_rep, rep_coord_lines in enumerate(replica_coord_line_sets):
        positions = np.array(rep_coord_lines, np.float64)
        ase_atoms = ase.Atoms(symbols=element_list, positions=positions, cell=cell)
        structure_data_list.append(StructureData(ase=ase_atoms))
        
    return structure_data_list

def neb_energies_and_distances_from_output(output_file_path):
    content = open(output_file_path).read()
    final_energy_str = re.findall(r'ENERGIES \[au\] = (.*?)BAND', content, re.DOTALL)[-1]
    energies = np.array(final_energy_str.split(),dtype=float) * scipy.constants.physical_constants['Hartree energy in eV'][0]
    energies -= energies[0]
    
    final_dist_str = re.findall(r'DISTANCES REP = (.*?)ENERGIES', content, re.DOTALL)[-1]
    distances = np.array(['0.0'] + final_dist_str.split(),dtype=float) * 0.529177249
    
    return distances, energies

In [ ]:
def make_replica_html(structure_data_list, energies):
    html = '<table>'
    
    for i, (rep, en) in enumerate(zip(structure_data_list, energies)):
        thumbnail = rep.get_extra('thumbnail')
        # The table cell
        if i%2 == 0:
            html += '<tr>'
        html += '<td><img width="400px" src="data:image/png;base64,{}" title="">'.format(thumbnail)
        # Output some information about the replica...
        html += '<p><b>Nr: </b>{} <br> <b>Energy:</b> {}</p>'\
                .format(i, en)
        html += '<p>pk: {}</p>'.format(rep.pk)
        # ... and the download link.
        html += '<p><a target="_blank" href="./export_structure.ipynb?uuid={}">Export Structure</a></p><td>'\
                .format(rep.uuid)
        if i%2 == 1:
            html += '</tr>'
            
    html += '</tr>'
    html += '</table>'
    return html

In [ ]:
def sorted_opt_rep_keys(keys):
    return sorted([ (int(key.split('_')[2]), key) for key in keys if 'opt_replica' in key])

def process_and_show_neb(c):
    
    btn_show.disabled = True
    with main_out:
        clear_output()
        
    
    wc = drop_nebs.value
    cp2k_calc = wc.get_outputs()[0]
    cc_output_dict = cp2k_calc.get_outputs_dict()

    #### --------------------------------------------------------------
    ## Create a list of structure_data object corresponding to replicas
    if 'opt_replica_0' in cc_output_dict:
        # NEW: get replicas from output nodes
        structure_data_list = []
        for rep_key in sorted_opt_rep_keys(cc_output_dict.keys()):
            if rep_key[0] >= len(structure_data_list):
                structure_data_list.append(cc_output_dict[rep_key[1]])
    else:
        # OLD: store replicas in nodes and link through extra
        structure_data_list = []
        for rep_key in sorted_opt_rep_keys(cp2k_calc.get_extras().keys()):
            struct_pk = cp2k_calc.get_extras()[rep_key[1]]
            structure_data_list.append(load_node(struct_pk))

        if len(structure_data_list) == 0:
            restart_file_path = cc_output_dict['retrieved'].get_abs_path('aiida-1.restart')
            structure_data_list = replicas_from_restart(restart_file_path)
            for i_rep, rep in enumerate(structure_data_list):
                rep.store()
                cp2k_calc.set_extra("opt_replica_%d" % i_rep, rep.pk)
                with main_out:
                    print("Added replica %d with pk %d to database" % (i_rep, rep.pk))
        
    #### --------------------------------------------------------------
    ## Add thumbnails to replicas if they are not already added
    for rep in structure_data_list:
        if not "thumbnail" in rep.get_extras():
            rep.set_extra("thumbnail", render_thumbnail(rep.get_ase()))
    
    #### --------------------------------------------------------------
    output_file_path = cc_output_dict['retrieved'].get_abs_path('aiida.out')
    distances, energies = neb_energies_and_distances_from_output(output_file_path)
    
    replica_html = make_replica_html(structure_data_list, energies)
    
    with main_out:
        plt.plot(energies, 'o-')
        plt.ylabel("Energy (eV)")
        plt.xlabel("Replica nr")
        plt.show()
        
        display(ipw.HTML(replica_html))
    
    btn_show.disabled = False

In [ ]:
qb = QueryBuilder()
qb.append(WorkCalculation,
          filters={
              'attributes._process_label': {'==': 'NEBWorkchain'}
          },
          tag='wc',
          project='*'
)
qb.append(Cp2kCalculation,
          output_of='wc',
          filters={
              'attributes.state': {'==': 'FINISHED'}}
)
qb.order_by({WorkCalculation: {'id': 'desc'}})
neb_wcs = qb.all()

In [ ]:
sel_options = OrderedDict([("PK %d: " % (neb_wc[0].pk) + neb_wc[0].description, neb_wc[0]) for neb_wc in neb_wcs])

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

drop_nebs = ipw.Dropdown(options = sel_options,
                            description = 'NEB: ', layout=layout, style=style)

btn_show = ipw.Button(description="Show")
btn_show.on_click(process_and_show_neb)

main_out = ipw.Output()

display(drop_nebs, btn_show, main_out)